In [96]:
import glob
import os
import json
import datetime
PROJECT_DIRECTORY = os.path.realpath(os.path.curdir)

In [97]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
import requests
from datetime import timedelta
from requests.auth import HTTPBasicAuth
from s3utils import save_to_s3, publish_to_s3, get_matching_s3_keys

In [99]:
S3_BUCKET_INTERNAL_NOTEBOOK_DATA = os.environ.get('S3_BUCKET_INTERNAL_NOTEBOOK_DATA')
S3_BUCKET_PUBLIC = os.environ.get('S3_BUCKET_PUBLIC')

# Get all seasons

In [100]:
seasons_response = requests.request(
            url="https://api.sportsnext.schibsted.io/v1/nor/tournaments/40/seasons",
            method="GET",
            headers={
                "Content-Type": "application/json; charset=utf-8",
                "User-Agent": "bt-bord4",
            })
seasons_json = json.loads(seasons_response.text)

In [101]:
def normalize_name(name):
    name = name.replace("IK Start", "Start")
    name = name.replace("Kongsvinger IL", "Kongsvinger")
    name = name.replace("Molde FK", "Molde")
    name = name.replace("Bodø / Glimt", "Bodø/Glimt")
    name = name.replace("Strømsgodset IF", "Strømsgodset")
    name = name.replace("Stabæk Fotball", "Stabæk")
    name = name.replace("Tromsø IL", "Tromsø")
    name = name.replace("Lillestrøm SK", "Lillestrøm")
    name = name.replace("Hødd IL", "Hødd")
    name = name.replace("Viking FK", "Viking")
    name = name.replace("Moss FK", "Moss")
    name = name.replace("Kristiansund BK", "Kristiansund")
    name = name.replace("Ranheim TF", "Ranheim")
    name = name.replace("Odd MAX", "Odd")
    name = name.replace("Sandefjord Fotball", "Sandefjord")
    name = name.replace("FK Haugesund", "Haugesund")
    name = name.replace("Mjøndalen IF", "Mjøndalen")
    name = name.replace("FK Eik-Tønsberg", "Eik-Tønsberg")
    name = name.replace("Sarpsborg FK", "Sarpsborg")
    name = name.replace("Tromsdalen UIL", "Tromsdalen")
    name = name.replace("FK Mandalskameratene", "Mandalskameratene")
    name = name.replace("FK Ørn-Horten", "Ørn-Horten")
    name = name.replace("Tollnes BK", "Tollnes")
    name = name.replace("FK Manglerud Star", "Manglerud Star")
    name = name.replace("Stavanger IF", "Stavanger")
    name = name.replace("Levanger FK", "Levanger")
    name = name.replace("KFUM Oslo", "KFUM")
    name = name.replace("Florø SK", "Florø")
    name = name.replace("Øygarden FK", "Øygarden")
    
    return name

In [102]:
def generate_season_data(data):
    matches = []
    
    participants = data["participants"]
    for event in data["events"]:
        if (event["tournament"]["phase"] == "group") & (event["status"]["type"] == "finished"):
            dt = datetime.datetime.strptime(event["startDate"], "%Y-%m-%dT%H:%M:%SZ")
            home_team=event["participantIds"][0]
            away_team=event["participantIds"][1]
            
            home_team_name = normalize_name(participants[home_team]["name"])
            away_team_name = normalize_name(participants[away_team]["name"])           
            
            if  (dt.year < 2022):
                result = "x"
                if "winnerId" in event["winners"]:
                    result = "1" if str(event["winners"]["winnerId"]) == str(home_team) else "2"
                
                home_goals_half_time = 0
                away_goals_half_time = 0
                if "period1Score" in event["results"][home_team]:
                    home_goals_half_time = event["results"][home_team]["period1Score"]
                    away_goals_half_time = event["results"][away_team]["period1Score"]
                else:
                    if "period2Score" in event["results"][home_team]:
                        home_goals_half_time = event["results"][home_team]["runningScore"] - event["results"][home_team]["period2Score"] 
                        away_goals_half_time = event["results"][away_team]["runningScore"] - event["results"][away_team]["period2Score"]
                match = {
                    "date": event["startDate"],
                    "year": dt.year,
                    "month": dt.month,
                    "day": dt.isoweekday(),
                    "_round": int(event["tournament"]["round"]), # the word round is reseverd in pandas, so add _ at the start
                    "home_team": home_team_name,
                    "away_team": away_team_name,
                    "home_goals_half_time": home_goals_half_time,
                    "away_goals_half_time": away_goals_half_time,
                    "home_goals_full_time": event["results"][home_team]["runningScore"],
                    "away_goals_full_time": event["results"][away_team]["runningScore"],
                    "result": result
                }

                match[home_team_name] = True
                match[away_team_name] = True
                matches.append(match)
                
    _df = pd.DataFrame.from_dict(matches)
    return _df

In [103]:
dfs = []
for season in seasons_json["tournamentSeasons"]:
    schedule_response = requests.request(
            url=f"https://api.sportsnext.schibsted.io/v1/nor/tournaments/seasons/{season['id']}/schedule",
            method="GET",
            headers={
                "Content-Type": "application/json; charset=utf-8",
                "User-Agent": "bt-bord4",
            })
    schedule_json = json.loads(schedule_response.text)
    print(season["name"])

    _df = generate_season_data(schedule_json)
    print(f"Data: {_df.shape}")
    dfs.append(_df)

1. Division 1997
Data: (182, 26)
1. Division 1998
Data: (182, 26)
1. Division 1999
Data: (182, 26)
1. Division 2000
Data: (182, 26)
1. Division 2001
Data: (240, 28)
1. Division 2002
Data: (240, 28)
1. Division 2003
Data: (240, 28)
1. Division 2004
Data: (240, 28)
Adeccoligaen 2005
Data: (240, 28)
Adeccoligaen 2006
Data: (240, 28)
Adeccoligaen 2007
Data: (240, 28)
Adeccoligaen 2008
Data: (240, 28)
Adeccoligaen 2009
Data: (240, 28)
Adeccoligaen 2010
Data: (210, 27)
Adeccoligaen 2011
Data: (240, 28)
1. Divisjon 2012
Data: (240, 28)
1. Divisjon 2013
Data: (240, 28)
1. Divisjon 2014
Data: (240, 28)
1. Divisjon 2015
Data: (240, 28)
1. Divisjon 2016
Data: (240, 28)
1. Divisjon 2017
Data: (240, 28)
1. Divisjon 2018
Data: (240, 28)
1. Divisjon 2019
Data: (240, 28)
1. Divisjon 2020
Data: (240, 28)
1. divisjon 2021
Data: (240, 28)
1. divisjon 2022
Data: (0, 0)


In [104]:
df = pd.concat(dfs)

In [105]:
df.home_team.unique()

array(['Aalesund', 'Bryne', 'Byåsen', 'Drøbak-Frogn', 'Odd', 'Runar',
       'Sarpsborg', 'HamKam', 'Start', 'Eik-Tønsberg', 'Harstad', 'Moss',
       'Vålerenga', 'Hødd', 'Lyn', 'Strindheim', 'Raufoss', 'Skeid',
       'Ullern', 'Kjelsås', 'Clausenengen', 'Haugesund', 'Hønefoss',
       'Sogndal', 'Lofoten', 'Skjetten', 'Kongsvinger', 'Sandefjord',
       'Strømsgodset', 'Tromsdalen', 'Mandalskameratene', 'Ørn-Horten',
       'Tollnes', 'Tromsø', 'Åsane', 'Lørenskog', 'Manglerud Star',
       'Alta', 'Bærum', 'Fredrikstad', 'Vard Haugesund', 'Pors', 'Follo',
       'Stabæk', 'Tønsberg', 'Løv-Ham', 'Bodø/Glimt', 'Sarpsborg 08',
       'Notodden', 'Molde', 'Sandnes Ulf', 'Nybergsund', 'Stavanger',
       'Mjøndalen', 'Strømmen', 'Ranheim', 'Randaberg', 'Asker',
       'Ullensaker/Kisa', 'Kristiansund', 'Elverum', 'Nest-Sotra', 'Jerv',
       'Levanger', 'Brann', 'KFUM', 'Florø', 'Arendal', 'Viking',
       'Øygarden', 'Grorud', 'Stjørdals/Blink', 'Lillestrøm'],
      dtype=object)

In [106]:
def calculate_ponts(x, team):
    if x.result == "x":
            return 1

    if x.home_team == team:
        if x.result == "1":
            return 3
        if x.result == "2":
            return 0
    
    if x.away_team == team:
        if x.result == "1":
            return 0
        if x.result == "2":
            return 3
    
    return -1

def find_home_away(x, team):
    if x.home_team == team:
        return "home"
    elif x.away_team == team:
        return "away"
    
    return "unknown"

def find_opponent_team(x, team):
    if x.home_team == team:
        return x.away_team
    elif x.away_team == team:
        return x.home_team

def team_goals(x, team):
    if x.home_team == team:
        return x.home_goals_full_time
    elif x.away_team == team:
        return x.away_goals_full_time

def team_conceded(x, team):
    if x.home_team == team:
        return x.away_goals_full_time
    elif x.away_team == team:
        return x.home_goals_full_time

def team_goals_half_time(x, team):
    if x.home_team == team:
        return x.home_goals_half_time
    elif x.away_team == team:
        return x.away_goals_half_time

def team_conceded_half_time(x, team):
    if x.home_team == team:
        return x.away_goals_half_time
    elif x.away_team == team:
        return x.home_goals_half_time
    

def non_win(x):
    if x.points == 3:
        return False
    
    return True

def non_loose(x):
    if x.points == 0:
        return False
    
    return True

def turn_around_win(x):
    if x.team_goals_half_time < x.team_conceded_half_time:
        if x.team_goals > x.team_conceded:
            return True
        else:
            return False
    
def turn_around_loose(x):
     if x.team_goals_half_time > x.team_conceded_half_time:
        if x.team_goals < x.team_conceded:
            return True
        else:
            return False

def half_time_full_time(x):
    result = ""
    if x.team_goals_half_time < x.team_conceded_half_time:
        result = "l"
    if x.team_goals_half_time == x.team_conceded_half_time:
        result = "d"
    if x.team_goals_half_time > x.team_conceded_half_time:
        result = "w"
    
    if x.team_goals < x.team_conceded:
        result += "l"
    if x.team_goals == x.team_conceded:
        result += "d"
    if x.team_goals > x.team_conceded:
        result += "w"
    
    return result

    

In [107]:
df["date"] = df["date"].astype("datetime64[ns]")

In [150]:
team_dfs = []
for team in df.home_team.unique():
    _df = df[df[team] == True].copy().reset_index()        
    _df["team"] = team
    _df = _df.sort_values('date', ascending=True)
    _df["opponent_team"] = _df.apply(lambda x: find_opponent_team(x, team), axis=1)
    _df["home_away"] = _df.apply(lambda x: find_home_away(x, team), axis=1)
    _df["season_game"] = _df.groupby(['year']).cumcount() + 1
    _df["points"]  = _df.apply(lambda x: calculate_ponts(x, team), axis=1)
    _df["team_goals"] = _df.apply(lambda x:team_goals(x, team),axis=1)
    _df["team_conceded"] = _df.apply(lambda x:team_conceded(x, team),axis=1)
    _df["team_goals_half_time"] = _df.apply(lambda x:team_goals_half_time(x, team),axis=1)
    _df["team_conceded_half_time"] = _df.apply(lambda x:team_conceded_half_time(x, team),axis=1)
    _df["non_win"] = _df.apply(lambda x:non_win(x),axis=1)
    _df["non_loose"] = _df.apply(lambda x:non_loose(x),axis=1)
    _df["turn_around_win"] = _df.apply(lambda x:turn_around_win(x),axis=1)
    _df["turn_around_loose"] = _df.apply(lambda x:turn_around_loose(x),axis=1)
    _df["half_time_full_time"] = _df.apply(lambda x:half_time_full_time(x),axis=1)
    
    _df["season_points_total"] = _df.groupby(['year'])['points'].apply(lambda x: x.cumsum())
    _df["season_points_avg"]  = _df["season_points_total"] / _df["season_game"]
    _df["season_points_avg_before"]  = _df["season_points_avg"].shift(1)
    _df["season_scored_total"] = _df.groupby(['year'])['team_goals'].apply(lambda x: x.cumsum())
    _df["season_conceded_total"] = _df.groupby(['year'])['team_conceded'].apply(lambda x: x.cumsum())
    
    _df.loc[_df.home_away == "home", "season_home_game"] = _df[_df.home_away == "home"].groupby(['year']).cumcount() + 1
    _df.loc[_df.home_away == "home", "season_home_points_total"] = _df[_df.home_away == "home"].groupby(['year'])['points'].apply(lambda x: x.cumsum())
    _df.loc[_df.home_away == "home", "season_home_points_avg"] = _df[_df.home_away == "home"]["season_home_points_total"] / _df[_df.home_away == "home"]["season_home_game"]
    _df.loc[_df.home_away == "home", "season_home_points_avg_before"]  = _df[_df.home_away == "home"]["season_home_points_avg"].shift(1)
    
    _df.loc[_df.home_away == "away", "season_away_game"] = _df[_df.home_away == "away"].groupby(['year']).cumcount() + 1
    _df.loc[_df.home_away == "away", "season_away_points_total"] = _df[_df.home_away == "away"].groupby(['year'])['points'].apply(lambda x: x.cumsum())
    _df.loc[_df.home_away == "away", "season_away_points_avg"] = _df[_df.home_away == "away"]["season_away_points_total"] / _df[_df.home_away == "away"]["season_away_game"]
    _df.loc[_df.home_away == "away", "season_away_points_avg_before"]  = _df[_df.home_away == "away"]["season_away_points_avg"].shift(1)
    
    _df["form_6_avg"] = _df["points"].rolling(6).mean()
    _df["form_30_avg"] = _df["points"].rolling(30).mean()
    
    _df_home = _df[_df.home_away == "home"] 
    _df_away = _df[_df.home_away == "away"] 
    

    _df = _df.sort_values('date', ascending=False)
    _df["season_game_reverse"] = _df.groupby(['year']).cumcount() + 1
    _df["season_points_total_reverse"] = _df.groupby(['year'])['points'].apply(lambda x: x.cumsum())
    _df["season_points_avg_reverse"]  = _df["season_points_total_reverse"] / _df["season_game_reverse"]
    _df["season_points_avg_after"]  = _df["season_points_avg_reverse"].shift(1)
    _df["season_scored_total_reverse"] = _df.groupby(['year'])['team_goals'].apply(lambda x: x.cumsum())
    _df["season_conceded_total_reverse"] = _df.groupby(['year'])['team_conceded'].apply(lambda x: x.cumsum())
    
    _df.loc[_df.home_away == "home", "season_home_game_reverse"] = _df[_df.home_away == "home"].groupby(['year']).cumcount() + 1
    _df.loc[_df.home_away == "home", "season_home_points_total_reverse"] = _df[_df.home_away == "home"].groupby(['year'])['points'].apply(lambda x: x.cumsum())
    _df.loc[_df.home_away == "home", "season_home_points_avg_reverse"] = _df[_df.home_away == "home"]["season_home_points_total_reverse"] / _df[_df.home_away == "home"]["season_home_game_reverse"]
    _df.loc[_df.home_away == "home", "season_home_points_avg_after"]  = _df[_df.home_away == "home"]["season_home_points_avg_reverse"].shift(1)
    
    _df.loc[_df.home_away == "away", "season_away_game_reverse"] = _df[_df.home_away == "away"].groupby(['year']).cumcount() + 1
    _df.loc[_df.home_away == "away", "season_away_points_total_reverse"] = _df[_df.home_away == "away"].groupby(['year'])['points'].apply(lambda x: x.cumsum())
    _df.loc[_df.home_away == "away", "season_away_points_avg_reverse"] = _df[_df.home_away == "away"]["season_away_points_total_reverse"] / _df[_df.home_away == "away"]["season_away_game_reverse"]
    _df.loc[_df.home_away == "away", "season_away_points_avg_after"]  = _df[_df.home_away == "away"]["season_away_points_avg_reverse"].shift(1)
    
    _df = _df.sort_values('_round', ascending=True)
    _df["season_round_points_total"] = _df.groupby(['year'])['points'].apply(lambda x: x.cumsum())
    _df["season_round_points_avg"]  = _df["season_round_points_total"] / _df["_round"]
    _df["season_round_scored_total"] = _df.groupby(['year'])['team_goals'].apply(lambda x: x.cumsum())
    _df["season_round_conceded_total"] = _df.groupby(['year'])['team_conceded'].apply(lambda x: x.cumsum())
    
    team_dfs.append(_df)

In [151]:
df_teams = pd.concat(team_dfs)
df_teams = df_teams.reset_index(drop=True)

In [152]:
standings_dfs = []
for season in seasons_json["tournamentSeasons"]:
    standings = []
    standing_response = requests.request(
            url=f"https://api.sportsnext.schibsted.io/v1/nor/tournaments/seasons/{season['id']}/standings",
            method="GET",
            headers={
                "Content-Type": "application/json; charset=utf-8",
                "User-Agent": "bt-bord4",
            })
    standing_json = json.loads(standing_response.text)
    participants = standing_json["participants"]
    for standing in standing_json["standings"][0]["teamStandings"]:
        # Standings returns Øygarden in stead of Nest-Sotra for 2018
        
        team_name = normalize_name(participants[standing["teamId"]]["name"])
        if int(standing_json["tournamentSeason"]["year"]) == 2018:
            if team_name == "Øygarden":
                team_name = "Nest-Sotra"
                
        standings.append(
            {
                "year": int(standing_json["tournamentSeason"]["year"]),
                "team": team_name,
                "season_final_rank": int(standing["rank"]),
                "season_final_rule": standing["rule"]["code"] if "rule" in standing else "",
                "season_final_games": standing["played"],
                "season_final_points": standing["points"],                
                "season_final_wins": standing["wins"],
                "season_final_draws": standing["draws"],
                "season_final_losses": standing["losses"],
                "season_final_goals_for": standing["goalsFor"],
                "season_final_goals_against":standing["goalsAgainst"]
            }
        )
    standings_dfs.append(pd.DataFrame.from_dict(standings))
    
df_standing =  pd.concat(standings_dfs)
df_standing["season_final_avg"] = df_standing["season_final_points"] / df_standing["season_final_games"]
df_standing = df_standing.replace("relegation_round", "relegation_playoffs")
df_standing = df_standing.reset_index(drop=True)

In [153]:
df_standing[df_standing.year == 2018]

,year,team,season_final_rank,season_final_rule,season_final_games,season_final_points,season_final_wins,season_final_draws,season_final_losses,season_final_goals_for,season_final_goals_against,season_final_avg
328,2018,Viking,1,promotion,30,61,20,1,9,68,44,2.033333
329,2018,Mjøndalen,2,promotion,30,60,17,9,4,49,24,2.000000
330,2018,Aalesund,3,promotion_playoffs,30,59,18,5,7,58,31,1.966667
331,2018,Sogndal,4,promotion_playoffs,30,51,15,6,9,47,31,1.700000
332,2018,Ullensaker/Kisa,5,promotion_playoffs,30,43,11,10,9,59,49,1.433333
333,2018,Nest-Sotra,6,promotion_playoffs,30,43,12,7,11,43,41,1.433333
334,2018,Tromsdalen,7,,30,43,12,7,11,43,47,1.433333
335,2018,Kongsvinger,8,,30,42,12,6,12,59,49,1.400000
336,2018,HamKam,9,,30,42,12,6,12,46,44,1.400000
337,2018,Sandnes Ulf,10,,30,42,11,9,10,43,47,1.400000


In [154]:
df_overview = pd.merge(df_teams, df_standing, on=["year", "team"])

In [155]:
df_overview.to_csv("data/processed/historical_obos_games.csv", index=False)

In [156]:
df_overview.head(10)

,index,date,year,month,day,_round,home_team,away_team,home_goals_half_time,away_goals_half_time,home_goals_full_time,away_goals_full_time,result,Aalesund,Hødd,Bryne,Moss,Byåsen,Eik-Tønsberg,Drøbak-Frogn,HamKam,Odd,Start,Runar,Harstad,...,season_points_avg_after,season_scored_total_reverse,season_conceded_total_reverse,season_home_game_reverse,season_home_points_total_reverse,season_home_points_avg_reverse,season_home_points_avg_after,season_away_game_reverse,season_away_points_total_reverse,season_away_points_avg_reverse,season_away_points_avg_after,season_round_points_total,season_round_points_avg,season_round_scored_total,season_round_conceded_total,season_final_rank,season_final_rule,season_final_games,season_final_points,season_final_wins,season_final_draws,season_final_losses,season_final_goals_for,season_final_goals_against,season_final_avg
0,0,1997-04-20 16:00:00,1997,4,7,1,Aalesund,Hødd,4,2,4,2,1,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.280000,36,36,13.0,21.0,1.615385,1.500000,NaN,NaN,NaN,NaN,3,3.000000,4,2,10,,26,35,11,2,13,36,36,1.346154
1,9,1997-04-27 16:00:00,1997,4,7,2,Eik-Tønsberg,Aalesund,3,1,3,1,1,True,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,...,1.333333,32,34,NaN,NaN,NaN,NaN,13.0,14.0,1.076923,1.166667,3,1.500000,5,5,10,,26,35,11,2,13,36,36,1.346154
2,14,1997-05-04 16:00:00,1997,5,7,3,Aalesund,Start,1,3,1,3,2,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,...,1.391304,31,31,12.0,18.0,1.500000,1.636364,NaN,NaN,NaN,NaN,3,1.000000,6,8,10,,26,35,11,2,13,36,36,1.346154
3,25,1997-05-08 16:00:00,1997,5,4,4,Moss,Aalesund,1,0,1,0,1,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.454545,30,28,NaN,NaN,NaN,NaN,12.0,14.0,1.166667,1.272727,3,0.750000,6,9,10,,26,35,11,2,13,36,36,1.346154
4,34,1997-05-16 16:00:00,1997,5,5,5,Aalesund,Bryne,0,2,0,2,2,True,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.523810,30,27,11.0,18.0,1.636364,1.800000,NaN,NaN,NaN,NaN,3,0.600000,6,11,10,,26,35,11,2,13,36,36,1.346154
5,39,1997-05-19 16:00:00,1997,5,1,6,Odd,Aalesund,1,1,1,1,x,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,...,1.550000,30,25,NaN,NaN,NaN,NaN,11.0,14.0,1.272727,1.300000,4,0.666667,7,12,10,,26,35,11,2,13,36,36,1.346154
6,41,1997-05-25 16:00:00,1997,5,7,7,Aalesund,Byåsen,0,3,0,3,2,True,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.631579,29,24,10.0,18.0,1.800000,2.000000,NaN,NaN,NaN,NaN,4,0.571429,7,15,10,,26,35,11,2,13,36,36,1.346154
7,49,1997-06-01 16:00:00,1997,6,7,8,Aalesund,Vålerenga,1,2,1,2,2,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.722222,29,21,9.0,18.0,2.000000,2.250000,NaN,NaN,NaN,NaN,4,0.500000,8,17,10,,26,35,11,2,13,36,36,1.346154
8,58,1997-06-08 16:00:00,1997,6,7,9,Runar,Aalesund,3,1,3,1,1,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,...,1.823529,28,19,NaN,NaN,NaN,NaN,10.0,13.0,1.300000,1.444444,4,0.444444,9,20,10,,26,35,11,2,13,36,36,1.346154
9,62,1997-06-15 16:00:00,1997,6,7,10,Aalesund,Drøbak-Frogn,3,0,3,0,1,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,...,1.750000,27,16,8.0,18.0,2.250000,2.142857,NaN,NaN,NaN,NaN,7,0.700000,12,20,10,,26,35,11,2,13,36,36,1.346154


In [157]:
df_overview.drop(columns=["index"], inplace=True)

In [158]:
df_overview = df_overview.sort_values("date")

In [159]:
df_overview.to_csv("data/processed/historical_obos.csv", index=False)

In [160]:
df_overview[(df_overview.year > 2008) & (df_overview.year < 2022)][["team", "date", "year", "month", "day", "_round", "home_team", "away_team", "home_goals_half_time", "away_goals_half_time", "home_goals_full_time", "away_goals_full_time", "result", "season_points_avg", "season_points_avg_after", "season_final_rank", "season_final_avg","season_points_total", "season_final_rule", "season_scored_total", "season_conceded_total", "season_final_points"]]

,team,date,year,month,day,_round,home_team,away_team,home_goals_half_time,away_goals_half_time,home_goals_full_time,away_goals_full_time,result,season_points_avg,season_points_avg_after,season_final_rank,season_final_avg,season_points_total,season_final_rule,season_scored_total,season_conceded_total,season_final_points
4942,Kongsvinger,2009-04-05 16:00:00,2009,4,7,1,Kongsvinger,HamKam,1,2,3,2,1,3.000000,1.827586,3,1.866667,3,promotion_playoffs,3,2,56
9112,Stavanger,2009-04-05 16:00:00,2009,4,7,1,Alta,Stavanger,0,1,1,3,2,3.000000,0.896552,15,0.966667,3,relegation,3,1,29
5920,Tromsdalen,2009-04-05 16:00:00,2009,4,7,1,Tromsdalen,Haugesund,1,1,2,2,x,1.000000,1.310345,12,1.300000,1,,2,2,39
9142,Mjøndalen,2009-04-05 16:00:00,2009,4,7,1,Nybergsund,Mjøndalen,0,0,1,0,1,0.000000,1.344828,11,1.300000,0,,0,1,39
8546,Notodden,2009-04-05 16:00:00,2009,4,7,1,Notodden,Skeid,0,3,0,3,2,0.000000,1.000000,14,0.966667,0,relegation,0,3,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6637,Åsane,2021-11-27 14:00:00,2021,11,6,30,Raufoss,Åsane,0,0,1,1,x,1.333333,NaN,7,1.333333,40,,44,53,40
11355,Grorud,2021-11-27 14:00:00,2021,11,6,30,Grorud,HamKam,1,1,1,2,2,1.133333,NaN,13,1.133333,34,,45,59,34
11415,Stjørdals/Blink,2021-11-27 14:00:00,2021,11,6,30,Stjørdals/Blink,Aalesund,1,0,1,0,1,1.033333,NaN,14,1.033333,31,relegation_playoffs,32,50,31
8845,Sandnes Ulf,2021-11-27 14:00:00,2021,11,6,30,Start,Sandnes Ulf,0,1,2,2,x,1.300000,NaN,8,1.300000,39,,43,49,39


In [165]:
publish_to_s3('bt-board4','data/sport/brann',
           'historical_data_obos', 
           datetime.datetime.today().strftime('%Y-%m-%d'), df_overview[(df_overview.year > 2011) & (df_overview.year < 2022)][["team", "date", "year", "month", "day", "_round", "home_team", "away_team", "home_goals_full_time", "away_goals_full_time", "points", "season_round_points_total", "season_round_scored_total", "season_round_conceded_total", "season_round_points_avg", "season_points_total", "season_scored_total", "season_conceded_total", "season_points_avg", "season_points_avg_after", "season_final_points", 
 "season_final_wins", 
 "season_final_draws", 
 "season_final_losses", 
 "season_final_goals_for", 
 "season_final_goals_against", "season_final_rank", "season_final_avg", "season_final_rule", "season_final_points"]].to_csv(index=False),
           content_type='text/csv', file_extension='csv', skip_archive=True
          ) 

https://www.bt.no/data/sport/brann/historical_data_obos/historical_data_obos_latest.csv


In [164]:
list(df_overview.columns)

['date',
 'year',
 'month',
 'day',
 '_round',
 'home_team',
 'away_team',
 'home_goals_half_time',
 'away_goals_half_time',
 'home_goals_full_time',
 'away_goals_full_time',
 'result',
 'Aalesund',
 'Hødd',
 'Bryne',
 'Moss',
 'Byåsen',
 'Eik-Tønsberg',
 'Drøbak-Frogn',
 'HamKam',
 'Odd',
 'Start',
 'Runar',
 'Harstad',
 'Sarpsborg',
 'Vålerenga',
 'Kjelsås',
 'Lyn',
 'Raufoss',
 'Ullern',
 'Strindheim',
 'Skeid',
 'Lofoten',
 'Skjetten',
 'Clausenengen',
 'Haugesund',
 'Hønefoss',
 'Sogndal',
 'Kongsvinger',
 'Sandefjord',
 'Tromsdalen',
 'Strømsgodset',
 'Mandalskameratene',
 'Ørn-Horten',
 'Manglerud Star',
 'Tollnes',
 'Tromsø',
 'Åsane',
 'Lørenskog',
 'Alta',
 'Bærum',
 'Fredrikstad',
 'Pors',
 'Vard Haugesund',
 'Follo',
 'Løv-Ham',
 'Stabæk',
 'Tønsberg',
 'Bodø/Glimt',
 'Sarpsborg 08',
 'Notodden',
 'Molde',
 'Nybergsund',
 'Sandnes Ulf',
 'Stavanger',
 'Mjøndalen',
 'Strømmen',
 'Ranheim',
 'Randaberg',
 'Asker',
 'Ullensaker/Kisa',
 'Kristiansund',
 'Elverum',
 'Nest-Sotra'